In [1]:
import cv2
import sdc.detection.svm_classifier as svm
import sdc.detection.heatmap as h
import sdc.detection.object_detection as od
import sdc.detection.window as w

from moviepy.editor import VideoFileClip

In [2]:
# Initialize classifier
clf = svm.SVMClassifier()
clf.load()

# Create detector
detector = od.ObjectDetection(clf)

# Define interest of area which area of the image will be analyzed
detector.settings.xregion = (330, 640)
detector.settings.yregion = (200, 330)

# Parameter which scales HOG image
detector.settings.scale = 0.7

In [3]:
y1, y2 = detector.settings.yregion
x1, x2 = detector.settings.xregion
heatmap = h.DynamicHeatmap((330, 640), 2)
frames = 0

# Detected objects
objects =[]

# How many frames we are going to skip
skip_frames = 10

# Function which will handle each frame
def process_frame(im):
    global frames
    global objects
    global skip_frames
    
    # Resize image by half
    img = cv2.resize(im, (im.shape[1]//2, im.shape[0]//2))
    if frames % skip_frames == 0:
        result, boxes = detector.find_objects(img)
        _ = heatmap.add_heat(boxes)    
        objects, mask, labels = heatmap.find_heat_rectangles(0, expand=(3, 1))
        
    img = w.draw_boxes(im, objects, thick=2, mult=2)
    frames += 1
    return img


In [4]:
video_output = 'videos/project_video_labelled.mp4'
clip1 = VideoFileClip("videos/project_video.mp4")
out_clip = clip1.fl_image(process_frame)
%time out_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video videos/project_video_labelled.mp4
[MoviePy] Writing video videos/project_video_labelled.mp4


100%|█████████▉| 1260/1261 [00:36<00:00, 31.57it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: videos/project_video_labelled.mp4 

CPU times: user 3min 19s, sys: 5.13 s, total: 3min 24s
Wall time: 36.8 s
